### Getting tables with means and standard deviations

In [ ]:
import numpy as np
import pandas as pd

filename = "results.csv"

# Training dataset size
m = 50000

df = pd.read_csv(filename).fillna(method="ffill")
# Separate correlations from the rest, since we don't compute means/std for them.
cor_df = df[df["Space"] == "correlation_space"]
df = df[df["Space"] != "correlation_space"].drop(columns=["Space"])


# Dataframe with the complexity measures (i.e. big values).
name_seed = df["Model"].str.rsplit('_', n=1)
df["Model_Name"] = name_seed.str[0]
df.drop(columns=["Model"], inplace=True)



# Dataframe with the smaller values (i.e, instead of the `init` and `orig` complexity measures, we compute the actual generalization bounds).
def get_generalization_bound(value):
    return np.sqrt(value / (2*(m-1)))

complexity_metrics_to_change = ["sharpness_init", "sharpness_orig", "sharpness_mag_init", "sharpness_mag_orig", "flatness_init", "flatness_orig", "flatness_mag_init", "flatness_mag_orig"]
df_gen_bounds = df.copy(deep=True)

df_gen_bounds['Value'] = df_gen_bounds.apply(lambda row: get_generalization_bound(row['Value']) if row['Metric'] in complexity_metrics_to_change else row['Value'], axis=1)
df_gen_bounds['Metric'] = df_gen_bounds['Metric'].apply(lambda x: 'bound_' + x if x in complexity_metrics_to_change else x)

In [ ]:
grouped_df = df.groupby(["Model_Name", "Metric"])["Value"].agg(["mean", "std"]).round(3)
grouped_df.to_csv("results_avg_std.csv")
grouped_df.head(20)

In [ ]:
# Alternative presentation of the results
unstacked_df = grouped_df.unstack(level='Model_Name')
unstacked_df.columns = unstacked_df.columns.reorder_levels([1, 0])
unstacked_df = unstacked_df.sort_index(axis=1)
unstacked_df.head(20)

In [ ]:
grouped_bounds_df = df_gen_bounds.groupby(["Model_Name", "Metric"])["Value"].agg(["mean", "std"]).round(3)
grouped_df.to_csv("results_bounds.csv")
grouped_bounds_df.head(20)

In [ ]:
# Alternative presentation of the results
unstacked_bound_df = grouped_bounds_df.unstack(level='Model_Name')
unstacked_bound_df.columns = unstacked_bound_df.columns.reorder_levels([1, 0])
unstacked_bound_df = unstacked_bound_df.sort_index(axis=1)
unstacked_bound_df.head(20)

### Copy the latex table to clipboard

In [ ]:
import subprocess

# Pick the latex table you want to copy.
latex_output = unstacked_df.to_latex()
# When using Mac, we can copy directly to clipboard.
subprocess.run('pbcopy', universal_newlines=True, input=latex_output)